### DEPENDENCY

In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS
from collections import Counter

## Convert JSON to CSV for games.json => steam 1

In [2]:
json_path = "data/games.json"

In [3]:
df = pd.read_json(json_path)
df = df.T
df.head()

,name,release_date,required_age,price,dlc_count,detailed_description,about_the_game,short_description,reviews,header_image,...,positive,negative,estimated_owners,average_playtime_forever,average_playtime_2weeks,median_playtime_forever,median_playtime_2weeks,peak_ccu,tags,discount
20200,Galactic Bowling,"Oct 21, 2008",0,19.99,0,Galactic Bowling is an exaggerated and stylize...,Galactic Bowling is an exaggerated and stylize...,Galactic Bowling is an exaggerated and stylize...,,https://cdn.akamai.steamstatic.com/steam/apps/...,...,6,11,0 - 20000,0,0,0,0,0,"{'Indie': 22, 'Casual': 21, 'Sports': 21, 'Bow...",NaN
655370,Train Bandit,"Oct 12, 2017",0,0.99,0,THE LAW!! Looks to be a showdown atop a train....,THE LAW!! Looks to be a showdown atop a train....,THE LAW!! Looks to be a showdown atop a train....,,https://cdn.akamai.steamstatic.com/steam/apps/...,...,53,5,0 - 20000,0,0,0,0,0,"{'Indie': 109, 'Action': 103, 'Pixel Graphics'...",NaN
1732930,Jolt Project,"Nov 17, 2021",0,4.99,0,Jolt Project: The army now has a new robotics ...,Jolt Project: The army now has a new robotics ...,"Shoot vehicles, blow enemies with a special at...",,https://cdn.akamai.steamstatic.com/steam/apps/...,...,0,0,0 - 20000,0,0,0,0,0,[],NaN
1355720,Henosis™,"Jul 23, 2020",0,5.99,0,HENOSIS™ is a mysterious 2D Platform Puzzler w...,HENOSIS™ is a mysterious 2D Platform Puzzler w...,HENOSIS™ is a mysterious 2D Platform Puzzler w...,,https://cdn.akamai.steamstatic.com/steam/apps/...,...,3,0,0 - 20000,0,0,0,0,0,"{'2D Platformer': 161, 'Atmospheric': 154, 'Su...",NaN
1139950,Two Weeks in Painland,"Feb 3, 2020",0,0.0,0,ABOUT THE GAME Play as a hacker who has arrang...,ABOUT THE GAME Play as a hacker who has arrang...,Two Weeks in Painland is a story-driven game a...,,https://cdn.akamai.steamstatic.com/steam/apps/...,...,50,8,0 - 20000,0,0,0,0,0,"{'Indie': 42, 'Adventure': 41, 'Nudity': 22, '...",NaN


app_id read as index, let's fix the column name

In [4]:
column_name_dict = {
"name": "Name",
"release_date": "Release date",
"required_age": "Required age",
"price": "Price",
"dlc_count": "DLC count",
"detailed_description": "Detailed description",
"about_the_game": "About the game",
"short_description": "Short description",
"reviews": "Reviews",
"header_image": "Header image",
"website": "Website",
"support_url": "Support url",
"support_email": "Support email",
"windows": "Windows",
"mac": "Mac",
"linux": "Linux",
"metacritic_score": "Metacritic score",
"metacritic_url": "Metacritic url",
"achievements": "Achievements",
"recommendations": "Recommendations",
"notes": "Notes",
"supported_languages": "Supported languages",
"full_audio_languages": "Full audio languages",
"packages": "Packages",
"developers": "Developers",
"publishers": "Publishers",
"categories": "Categories",
"genres": "Genres",
"screenshots": "Screenshots",
"movies": "Movies",
"user_score": "User score",
"score_rank": "Score rank",
"positive": "Positive",
"negative": "Negative",
"estimated_owners": "Estimated owners",
"average_playtime_forever": "Average playtime forever",
"average_playtime_2weeks": "Average playtime two weeks",
"median_playtime_forever": "Median playtime forever",
"median_playtime_2weeks": "Median playtime two weeks",
"peak_ccu": "Peak CCU",
"tags": "Tags"
}

In [10]:
def convert_dict_to_string(dict_object):
    if len(dict_object) == 0:
        return np.NaN
    key_list = list(dict_object.keys())
    keys_string = ",".join(key_list)
    return keys_string

def read_convert_json_dataset():
    _df = pd.read_json(json_path)
    _df = _df.T
    _df['app_id'] = _df.index
    _df.rename(columns=column_name_dict, inplace=True)

    for col in _df.columns.values:
        if isinstance(_df.loc[546560][col], dict):
            _df[col] = _df[col].apply(lambda entries: convert_dict_to_string(entries))
        if isinstance(_df.loc[546560][col], list):
            if isinstance(_df.loc[546560][col][0], str):
                _df[col] = _df[col].apply(lambda entries: ",".join(entries))
    _df = _df.reset_index().set_index("app_id")
    return _df

df = read_convert_json_dataset()

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 111452 entries, 20200 to 3183790
Data columns (total 43 columns):
 #   Column                      Non-Null Count   Dtype 
---  ------                      --------------   ----- 
 0   index                       111452 non-null  int64 
 1   Name                        111452 non-null  object
 2   Release date                111452 non-null  object
 3   Required age                111452 non-null  object
 4   Price                       111452 non-null  object
 5   DLC count                   111452 non-null  object
 6   Detailed description        111452 non-null  object
 7   About the game              111452 non-null  object
 8   Short description           111452 non-null  object
 9   Reviews                     111452 non-null  object
 10  Header image                111452 non-null  object
 11  Website                     111452 non-null  object
 12  Support url                 111452 non-null  object
 13  Support email           

In [12]:
df.head()

,index,Name,Release date,Required age,Price,DLC count,Detailed description,About the game,Short description,Reviews,...,Positive,Negative,Estimated owners,Average playtime forever,Average playtime two weeks,Median playtime forever,Median playtime two weeks,Peak CCU,Tags,discount
app_id,,,,,,,,,,,,,,,,,,,,,
20200,20200,Galactic Bowling,"Oct 21, 2008",0,19.99,0,Galactic Bowling is an exaggerated and stylize...,Galactic Bowling is an exaggerated and stylize...,Galactic Bowling is an exaggerated and stylize...,,...,6,11,0 - 20000,0,0,0,0,0,"Indie,Casual,Sports,Bowling",NaN
655370,655370,Train Bandit,"Oct 12, 2017",0,0.99,0,THE LAW!! Looks to be a showdown atop a train....,THE LAW!! Looks to be a showdown atop a train....,THE LAW!! Looks to be a showdown atop a train....,,...,53,5,0 - 20000,0,0,0,0,0,"Indie,Action,Pixel Graphics,2D,Retro,Arcade,Sc...",NaN
1732930,1732930,Jolt Project,"Nov 17, 2021",0,4.99,0,Jolt Project: The army now has a new robotics ...,Jolt Project: The army now has a new robotics ...,"Shoot vehicles, blow enemies with a special at...",,...,0,0,0 - 20000,0,0,0,0,0,NaN,NaN
1355720,1355720,Henosis™,"Jul 23, 2020",0,5.99,0,HENOSIS™ is a mysterious 2D Platform Puzzler w...,HENOSIS™ is a mysterious 2D Platform Puzzler w...,HENOSIS™ is a mysterious 2D Platform Puzzler w...,,...,3,0,0 - 20000,0,0,0,0,0,"2D Platformer,Atmospheric,Surreal,Mystery,Puzz...",NaN
1139950,1139950,Two Weeks in Painland,"Feb 3, 2020",0,0.0,0,ABOUT THE GAME Play as a hacker who has arrang...,ABOUT THE GAME Play as a hacker who has arrang...,Two Weeks in Painland is a story-driven game a...,,...,50,8,0 - 20000,0,0,0,0,0,"Indie,Adventure,Nudity,Violent,Sexual Content,...",NaN


In [14]:
df.drop(columns=['index'], inplace=True)

In [15]:
df.to_csv("data/converted_big.csv") # Saving the converted dataframe
converted = pd.read_csv("data/converted_big.csv") # Reading it back
converted.head()

,app_id,Name,Release date,Required age,Price,DLC count,Detailed description,About the game,Short description,Reviews,...,Positive,Negative,Estimated owners,Average playtime forever,Average playtime two weeks,Median playtime forever,Median playtime two weeks,Peak CCU,Tags,discount
0,20200,Galactic Bowling,"Oct 21, 2008",0,19.99,0,Galactic Bowling is an exaggerated and stylize...,Galactic Bowling is an exaggerated and stylize...,Galactic Bowling is an exaggerated and stylize...,NaN,...,6,11,0 - 20000,0,0,0,0,0,"Indie,Casual,Sports,Bowling",NaN
1,655370,Train Bandit,"Oct 12, 2017",0,0.99,0,THE LAW!! Looks to be a showdown atop a train....,THE LAW!! Looks to be a showdown atop a train....,THE LAW!! Looks to be a showdown atop a train....,NaN,...,53,5,0 - 20000,0,0,0,0,0,"Indie,Action,Pixel Graphics,2D,Retro,Arcade,Sc...",NaN
2,1732930,Jolt Project,"Nov 17, 2021",0,4.99,0,Jolt Project: The army now has a new robotics ...,Jolt Project: The army now has a new robotics ...,"Shoot vehicles, blow enemies with a special at...",NaN,...,0,0,0 - 20000,0,0,0,0,0,NaN,NaN
3,1355720,Henosis™,"Jul 23, 2020",0,5.99,0,HENOSIS™ is a mysterious 2D Platform Puzzler w...,HENOSIS™ is a mysterious 2D Platform Puzzler w...,HENOSIS™ is a mysterious 2D Platform Puzzler w...,NaN,...,3,0,0 - 20000,0,0,0,0,0,"2D Platformer,Atmospheric,Surreal,Mystery,Puzz...",NaN
4,1139950,Two Weeks in Painland,"Feb 3, 2020",0,0.00,0,ABOUT THE GAME Play as a hacker who has arrang...,ABOUT THE GAME Play as a hacker who has arrang...,Two Weeks in Painland is a story-driven game a...,NaN,...,50,8,0 - 20000,0,0,0,0,0,"Indie,Adventure,Nudity,Violent,Sexual Content,...",NaN


## EDA Steam1

### READ DATA

In [16]:
# read converted big
df = pd.read_csv("data/converted_big.csv")
df.head(2)

,app_id,Name,Release date,Required age,Price,DLC count,Detailed description,About the game,Short description,Reviews,...,Positive,Negative,Estimated owners,Average playtime forever,Average playtime two weeks,Median playtime forever,Median playtime two weeks,Peak CCU,Tags,discount
0,20200,Galactic Bowling,"Oct 21, 2008",0,19.99,0,Galactic Bowling is an exaggerated and stylize...,Galactic Bowling is an exaggerated and stylize...,Galactic Bowling is an exaggerated and stylize...,NaN,...,6,11,0 - 20000,0,0,0,0,0,"Indie,Casual,Sports,Bowling",NaN
1,655370,Train Bandit,"Oct 12, 2017",0,0.99,0,THE LAW!! Looks to be a showdown atop a train....,THE LAW!! Looks to be a showdown atop a train....,THE LAW!! Looks to be a showdown atop a train....,NaN,...,53,5,0 - 20000,0,0,0,0,0,"Indie,Action,Pixel Graphics,2D,Retro,Arcade,Sc...",NaN


### BASIC INFO

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 111452 entries, 0 to 111451
Data columns (total 43 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   app_id                      111452 non-null  int64  
 1   Name                        111446 non-null  object 
 2   Release date                111452 non-null  object 
 3   Required age                111452 non-null  int64  
 4   Price                       111452 non-null  float64
 5   DLC count                   111452 non-null  int64  
 6   Detailed description        104995 non-null  object 
 7   About the game              104969 non-null  object 
 8   Short description           105076 non-null  object 
 9   Reviews                     10624 non-null   object 
 10  Header image                111452 non-null  object 
 11  Website                     46458 non-null   object 
 12  Support url                 50769 non-null   object 
 13  Support email 

In [18]:
df.columns

Index(['app_id', 'Name', 'Release date', 'Required age', 'Price', 'DLC count',
       'Detailed description', 'About the game', 'Short description',
       'Reviews', 'Header image', 'Website', 'Support url', 'Support email',
       'Windows', 'Mac', 'Linux', 'Metacritic score', 'Metacritic url',
       'Achievements', 'Recommendations', 'Notes', 'Supported languages',
       'Full audio languages', 'Packages', 'Developers', 'Publishers',
       'Categories', 'Genres', 'Screenshots', 'Movies', 'User score',
       'Score rank', 'Positive', 'Negative', 'Estimated owners',
       'Average playtime forever', 'Average playtime two weeks',
       'Median playtime forever', 'Median playtime two weeks', 'Peak CCU',
       'Tags', 'discount'],
      dtype='object')

In [19]:
df.shape

(111452, 43)

### MISSING VALUE IDENTIFICATION

In [21]:
# Get missing value amount and percentage for each column and sort Descending
missing_values = df.isna().sum()
missing_values = missing_values[missing_values > 0]
missing_values = pd.DataFrame(missing_values)
missing_values.columns = ['Missing Values']
missing_values['Percentage'] = (missing_values['Missing Values'] / df.shape[0]) * 100
missing_values = missing_values.sort_values(by='Missing Values', ascending=False)
missing_values

,Missing Values,Percentage
Score rank,111408,99.960521
Metacritic url,107447,96.406525
Reviews,100828,90.467645
discount,97410,87.400854
Notes,93002,83.445788
Full audio languages,65823,59.059505
Website,64994,58.315687
Support url,60683,54.447655
Tags,37423,33.577684
Support email,19022,17.067437


- Beberapa game di steam ada yang dibuat untuk testing jadi bisa aja gapunya score rank
- No metacritic url bisa jadi karena emang ga diprovide linknya, tapi metacritic scorenya ada semua kok dan ini yang lebih penting
- No review bisa jadi karena gamenya untuk testing atau datanya rusak ketika diambil dari source
- discount: game dijual tanpa diskon, make sense aja apalagi kalo gamenya juga yang gratisan, ini diliar dlu lagi aja, drop belakangan soalnya bisa dipake buat cari insight diskon-price-estimated owners
- Notes: gaada notes tambahan terkait gamenya, normal aja
- Full audio langauges: game tanpa dialog, normal aja
- Website: gamenya dirilis lgsg ke steam tanpa link webnya, normal aja
- Support url: gaada url utk help/customer support dari gamenya
- Tags: tag dikasih sama user ke game, kalo gamenya baru rilis atau sepi/gaada pemain ya gaada tagnya
- Movies: game dirilis tanpa ada film, bisa aja berguna karena kadang ada yang download/tau game dari filmnya gitu
- Support email: yaudah gaada, jarang pengaruh jg rasanya
- Kategori: game tanpa kategori, ini bikin susah sih mungkin akan didrop row yg kategorinya kosong
- Publishers: ini dapat berpengaruh
- About the game: gaada deskripsi, gpp sih
- Developers: ini juga bisa pengaruh ke gamenya
- Detail desc: ya gt
- Genres: penting sih, bakalan di drop row yang null genrenya
- Supported languages: bisa berpengaruh
- Short desc: ya gt
- Screenshots: berupa link url ss nya
- Name: game tanpa nama, gamungkin gasi?

In [22]:
# Explore some columns
df['Website']

0              http://www.galacticbowling.net
1                      http://trainbandit.com
2                                         NaN
3                    https://henosisgame.com/
4         https://www.unusual-games.com/home/
                         ...                 
111447                                    NaN
111448         https://www.hentaipanda.games/
111449                                    NaN
111450                   https://accubow.com/
111451               https://www.ledxfun.com/
Name: Website, Length: 111452, dtype: object

In [23]:
df['Header image']

0         https://cdn.akamai.steamstatic.com/steam/apps/...
1         https://cdn.akamai.steamstatic.com/steam/apps/...
2         https://cdn.akamai.steamstatic.com/steam/apps/...
3         https://cdn.akamai.steamstatic.com/steam/apps/...
4         https://cdn.akamai.steamstatic.com/steam/apps/...
                                ...                        
111447    https://shared.akamai.steamstatic.com/store_it...
111448    https://shared.akamai.steamstatic.com/store_it...
111449    https://shared.akamai.steamstatic.com/store_it...
111450    https://shared.akamai.steamstatic.com/store_it...
111451    https://shared.akamai.steamstatic.com/store_it...
Name: Header image, Length: 111452, dtype: object

In [24]:
# Drop column that is an image or url
df.drop(columns=['Header image', 'Website', 'Support url', 'Support email', 'Metacritic url'], inplace=True)

In [26]:
df.columns

Index(['app_id', 'Name', 'Release date', 'Required age', 'Price', 'DLC count',
       'Detailed description', 'About the game', 'Short description',
       'Reviews', 'Windows', 'Mac', 'Linux', 'Metacritic score',
       'Achievements', 'Recommendations', 'Notes', 'Supported languages',
       'Full audio languages', 'Packages', 'Developers', 'Publishers',
       'Categories', 'Genres', 'Screenshots', 'Movies', 'User score',
       'Score rank', 'Positive', 'Negative', 'Estimated owners',
       'Average playtime forever', 'Average playtime two weeks',
       'Median playtime forever', 'Median playtime two weeks', 'Peak CCU',
       'Tags', 'discount'],
      dtype='object')

I think its enough since we have other dataset that needs exploration

In [28]:
# put new dataframe into steam1 variable
steam1 = df.copy()

## EDA STEAM 2

In [29]:
steam2 = pd.read_csv('data/games_small.csv')
steam2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50872 entries, 0 to 50871
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   app_id          50872 non-null  int64  
 1   title           50872 non-null  object 
 2   date_release    50872 non-null  object 
 3   win             50872 non-null  bool   
 4   mac             50872 non-null  bool   
 5   linux           50872 non-null  bool   
 6   rating          50872 non-null  object 
 7   positive_ratio  50872 non-null  int64  
 8   user_reviews    50872 non-null  int64  
 9   price_final     50872 non-null  float64
 10  price_original  50872 non-null  float64
 11  discount        50872 non-null  float64
 12  steam_deck      50872 non-null  bool   
dtypes: bool(4), float64(3), int64(3), object(3)
memory usage: 3.7+ MB


Here we found other columns: rating, positive_ratio, user_reviews, price_original, price_final. Lets explore this new features

In [30]:
steam2.head()

,app_id,title,date_release,win,mac,linux,rating,positive_ratio,user_reviews,price_final,price_original,discount,steam_deck
0,13500,Prince of Persia: Warrior Within™,2008-11-21,True,False,False,Very Positive,84,2199,9.99,9.99,0.0,True
1,22364,BRINK: Agents of Change,2011-08-03,True,False,False,Positive,85,21,2.99,2.99,0.0,True
2,113020,Monaco: What's Yours Is Mine,2013-04-24,True,True,True,Very Positive,92,3722,14.99,14.99,0.0,True
3,226560,Escape Dead Island,2014-11-18,True,False,False,Mixed,61,873,14.99,14.99,0.0,True
4,249050,Dungeon of the ENDLESS™,2014-10-27,True,True,False,Very Positive,88,8784,11.99,11.99,0.0,True


app_id, title, date_release, win, mac, linux, and discount is similar as before, so we can drop these columns for further analysis

### Check whether the same app_id between steam1 and steam2 refer to the same game

In [31]:
steam1[steam1['app_id'] == 13500]

,app_id,Name,Release date,Required age,Price,DLC count,Detailed description,About the game,Short description,Reviews,...,Positive,Negative,Estimated owners,Average playtime forever,Average playtime two weeks,Median playtime forever,Median playtime two weeks,Peak CCU,Tags,discount
31285,13500,Prince of Persia: Warrior Within™,"Nov 21, 2008",17,9.99,0,Enter the dark underworld of Prince of Persia ...,Enter the dark underworld of Prince of Persia ...,Enter the dark underworld of Prince of Persia ...,NaN,...,1807,307,200000 - 500000,61,10,77,10,18,"Action,Adventure,Parkour,Third Person,Great So...",NaN


In [32]:
steam2[steam2['app_id'] == 13500]

,app_id,title,date_release,win,mac,linux,rating,positive_ratio,user_reviews,price_final,price_original,discount,steam_deck
0,13500,Prince of Persia: Warrior Within™,2008-11-21,True,False,False,Very Positive,84,2199,9.99,9.99,0.0,True


- We can conclude that steam1 and steam1 are intersecting, we can combine them using the app_id.
- The date_release in steam2 already in proper date format, we will use this column rather than the Release date in steam1
- Its better to use discount from the steam2 since its seems connected to the price_final and price_original
- We will remove Price from steam1 since it is not clear stating the final price or original price

In [33]:
# Drop release date, Price, and discount from steam1
steam1.drop(columns=['Release date', 'Price', 'discount'], inplace=True)
steam1.columns

Index(['app_id', 'Name', 'Required age', 'DLC count', 'Detailed description',
       'About the game', 'Short description', 'Reviews', 'Windows', 'Mac',
       'Linux', 'Metacritic score', 'Achievements', 'Recommendations', 'Notes',
       'Supported languages', 'Full audio languages', 'Packages', 'Developers',
       'Publishers', 'Categories', 'Genres', 'Screenshots', 'Movies',
       'User score', 'Score rank', 'Positive', 'Negative', 'Estimated owners',
       'Average playtime forever', 'Average playtime two weeks',
       'Median playtime forever', 'Median playtime two weeks', 'Peak CCU',
       'Tags'],
      dtype='object')

In [34]:
# Drop title, win, mac, linux from steam2
steam2.drop(columns=['title', 'win', 'mac', 'linux'], inplace=True)
steam2.columns

Index(['app_id', 'date_release', 'rating', 'positive_ratio', 'user_reviews',
       'price_final', 'price_original', 'discount', 'steam_deck'],
      dtype='object')

## MERGE steam1 and steam2

In [35]:
steam1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 111452 entries, 0 to 111451
Data columns (total 35 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   app_id                      111452 non-null  int64  
 1   Name                        111446 non-null  object 
 2   Required age                111452 non-null  int64  
 3   DLC count                   111452 non-null  int64  
 4   Detailed description        104995 non-null  object 
 5   About the game              104969 non-null  object 
 6   Short description           105076 non-null  object 
 7   Reviews                     10624 non-null   object 
 8   Windows                     111452 non-null  bool   
 9   Mac                         111452 non-null  bool   
 10  Linux                       111452 non-null  bool   
 11  Metacritic score            111452 non-null  int64  
 12  Achievements                111452 non-null  int64  
 13  Recommendation

In [36]:
steam2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50872 entries, 0 to 50871
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   app_id          50872 non-null  int64  
 1   date_release    50872 non-null  object 
 2   rating          50872 non-null  object 
 3   positive_ratio  50872 non-null  int64  
 4   user_reviews    50872 non-null  int64  
 5   price_final     50872 non-null  float64
 6   price_original  50872 non-null  float64
 7   discount        50872 non-null  float64
 8   steam_deck      50872 non-null  bool   
dtypes: bool(1), float64(3), int64(3), object(2)
memory usage: 3.2+ MB


In [37]:
# Merge on app_id
merged_df = pd.merge(steam1, steam2, on='app_id', how='inner')
merged_df

,app_id,Name,Required age,DLC count,Detailed description,About the game,Short description,Reviews,Windows,Mac,...,Peak CCU,Tags,date_release,rating,positive_ratio,user_reviews,price_final,price_original,discount,steam_deck
0,20200,Galactic Bowling,0,0,Galactic Bowling is an exaggerated and stylize...,Galactic Bowling is an exaggerated and stylize...,Galactic Bowling is an exaggerated and stylize...,NaN,True,False,...,0,"Indie,Casual,Sports,Bowling",2008-10-21,Mostly Negative,33,12,19.99,19.99,0.0,True
1,655370,Train Bandit,0,0,THE LAW!! Looks to be a showdown atop a train....,THE LAW!! Looks to be a showdown atop a train....,THE LAW!! Looks to be a showdown atop a train....,NaN,True,True,...,0,"Indie,Action,Pixel Graphics,2D,Retro,Arcade,Sc...",2017-10-12,Very Positive,88,51,0.27,0.99,72.0,True
2,1139950,Two Weeks in Painland,0,0,ABOUT THE GAME Play as a hacker who has arrang...,ABOUT THE GAME Play as a hacker who has arrang...,Two Weeks in Painland is a story-driven game a...,NaN,True,True,...,0,"Indie,Adventure,Nudity,Violent,Sexual Content,...",2020-02-03,Very Positive,86,58,0.00,0.00,0.0,True
3,1469160,Wartune Reborn,0,0,Feel tired of auto-fight? Feel tired of boring...,Feel tired of auto-fight? Feel tired of boring...,'Wartune Reborn' epic magic TBS game was devel...,NaN,True,False,...,68,"Turn-Based Combat,Massively Multiplayer,Multip...",2021-02-26,Mixed,63,168,0.00,0.00,0.0,True
4,1659180,TD Worlds,0,1,"TD Worlds is a dynamic, highly strategical gam...","TD Worlds is a dynamic, highly strategical gam...","TD Worlds is a dynamic, highly strategical gam...",NaN,True,False,...,3,"Tower Defense,Rogue-lite,RTS,Replay Value,Perm...",2022-01-09,Mixed,65,23,10.99,10.99,0.0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41165,1272080,PAYDAY 3,17,0,Step out of retirement back into the life of c...,Step out of retirement back into the life of c...,PAYDAY 3 is the much anticipated sequel to one...,NaN,True,False,...,3532,"Heist,Co-op,Action,Crime,FPS,Multiplayer,Shoot...",2023-09-21,Mostly Negative,38,29458,40.00,0.00,0.0,True
41166,2342950,God Of Weapons,0,0,⚔️CHECK THIS OUT!⚔️ / About the Game The world...,The world was once a bright and beautiful plac...,Embark on a thrilling journey in God of Weapon...,NaN,True,False,...,695,"Action Roguelike,RPG,Inventory Management,Rogu...",2023-09-12,Very Positive,84,1263,6.00,0.00,0.0,True
41167,1692240,Fortune's Run,0,1,"Discover the quiet dystopia of Fortune's Run, ...","Discover the quiet dystopia of Fortune's Run, ...",Explore a decaying industrial world left behin...,NaN,True,False,...,4,"Early Access,Cyberpunk,Immersive Sim,FPS,Steal...",2023-09-27,Very Positive,94,145,18.00,0.00,0.0,True
41168,1580180,Gunbrella,0,2,Gunbrella is a gritty noir-punk action-adventu...,Gunbrella is a gritty noir-punk action-adventu...,"Uncover the dark secrets of a gritty, tangled ...",“Gunbrella’s crunchy combat and excellent move...,True,False,...,19,"Side Scroller,Exploration,Platformer,Action Ro...",2023-09-13,Very Positive,88,248,15.00,0.00,0.0,True


In [38]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41170 entries, 0 to 41169
Data columns (total 43 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   app_id                      41170 non-null  int64  
 1   Name                        41170 non-null  object 
 2   Required age                41170 non-null  int64  
 3   DLC count                   41170 non-null  int64  
 4   Detailed description        41123 non-null  object 
 5   About the game              41113 non-null  object 
 6   Short description           41156 non-null  object 
 7   Reviews                     7920 non-null   object 
 8   Windows                     41170 non-null  bool   
 9   Mac                         41170 non-null  bool   
 10  Linux                       41170 non-null  bool   
 11  Metacritic score            41170 non-null  int64  
 12  Achievements                41170 non-null  int64  
 13  Recommendations             411

In [42]:
# Rename columns so it in same name format
merged_df.rename(columns={
    'app_id': 'App ID',
    'date_release': 'Release date',
    'positive_ratio': 'Positive ratio',
    'price_final': 'Price final',
    'price_original': 'Price original',
    'discount': 'Discount',
    'steam_deck': 'Steam Deck',
}, inplace=True)

In [43]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41170 entries, 0 to 41169
Data columns (total 43 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   App ID                      41170 non-null  int64  
 1   Name                        41170 non-null  object 
 2   Required age                41170 non-null  int64  
 3   DLC count                   41170 non-null  int64  
 4   Detailed description        41123 non-null  object 
 5   About the game              41113 non-null  object 
 6   Short description           41156 non-null  object 
 7   Reviews                     7920 non-null   object 
 8   Windows                     41170 non-null  bool   
 9   Mac                         41170 non-null  bool   
 10  Linux                       41170 non-null  bool   
 11  Metacritic score            41170 non-null  int64  
 12  Achievements                41170 non-null  int64  
 13  Recommendations             411

In [45]:
# show 10 first row of sorted id merged_df
merged_df = merged_df.sort_values(by='App ID', ascending=True)
merged_df.head(10)

,App ID,Name,Required age,DLC count,Detailed description,About the game,Short description,Reviews,Windows,Mac,...,Peak CCU,Tags,Release date,rating,Positive ratio,user_reviews,Price final,Price original,Discount,Steam Deck
22902,10,Counter-Strike,0,0,Play the world's number 1 online action game. ...,Play the world's number 1 online action game. ...,Play the world's number 1 online action game. ...,NaN,True,True,...,13230,"Action,FPS,Multiplayer,Shooter,Classic,Team-Ba...",2000-11-01,Overwhelmingly Positive,97,136118,9.99,9.99,0.0,True
11359,20,Team Fortress Classic,0,0,One of the most popular online action games of...,One of the most popular online action games of...,One of the most popular online action games of...,NaN,True,True,...,110,"Action,FPS,Multiplayer,Classic,Hero Shooter,Sh...",1999-04-01,Very Positive,85,5146,4.99,4.99,0.0,True
2120,30,Day of Defeat,0,0,Enlist in an intense brand of Axis vs. Allied ...,Enlist in an intense brand of Axis vs. Allied ...,Enlist in an intense brand of Axis vs. Allied ...,NaN,True,True,...,106,"FPS,World War II,Multiplayer,Shooter,Action,Wa...",2003-05-01,Very Positive,87,3489,4.99,4.99,0.0,True
19935,40,Deathmatch Classic,0,0,Enjoy fast-paced multiplayer gaming with Death...,Enjoy fast-paced multiplayer gaming with Death...,Enjoy fast-paced multiplayer gaming with Death...,NaN,True,True,...,2,"Action,FPS,Classic,Multiplayer,Shooter,First-P...",2001-06-01,Mostly Positive,79,1757,4.99,4.99,0.0,True
22696,50,Half-Life: Opposing Force,0,0,Return to the Black Mesa Research Facility as ...,Return to the Black Mesa Research Facility as ...,Return to the Black Mesa Research Facility as ...,NaN,True,True,...,120,"FPS,Action,Classic,Sci-fi,Singleplayer,Shooter...",1999-11-01,Overwhelmingly Positive,95,14057,4.99,4.99,0.0,True
20171,60,Ricochet,0,0,A futuristic action game that challenges your ...,A futuristic action game that challenges your ...,A futuristic action game that challenges your ...,NaN,True,True,...,10,"Action,FPS,Multiplayer,Classic,First-Person,Sc...",2000-11-01,Very Positive,80,3395,4.99,4.99,0.0,True
28761,70,Half-Life,0,1,Named Game of the Year by over 50 publications...,Named Game of the Year by over 50 publications...,Named Game of the Year by over 50 publications...,NaN,True,True,...,752,"FPS,Singleplayer,Sci-fi,Multiplayer,1990's,Act...",1998-11-08,Overwhelmingly Positive,96,72475,9.99,9.99,0.0,True
5803,80,Counter-Strike: Condition Zero,0,0,"With its extensive Tour of Duty campaign, a ne...","With its extensive Tour of Duty campaign, a ne...","With its extensive Tour of Duty campaign, a ne...",NaN,True,True,...,425,"Action,FPS,Shooter,Multiplayer,First-Person,Si...",2004-03-01,Very Positive,90,16187,9.99,9.99,0.0,True
29269,130,Half-Life: Blue Shift,0,0,Made by Gearbox Software and originally releas...,Made by Gearbox Software and originally releas...,Made by Gearbox Software and originally releas...,NaN,True,True,...,78,"FPS,Action,Sci-fi,Singleplayer,Shooter,Aliens,...",2001-06-01,Very Positive,91,10522,4.99,4.99,0.0,True
21380,220,Half-Life 2,0,1,1998. HALF-LIFE sends a shock through the game...,1998. HALF-LIFE sends a shock through the game...,1998. HALF-LIFE sends a shock through the game...,NaN,True,True,...,1015,"FPS,Action,Sci-fi,Classic,Singleplayer,Story R...",2007-10-10,Overwhelmingly Positive,97,122230,9.99,9.99,0.0,True


In [46]:
# Save result
merged_df.to_csv("data/merged_steam_data.csv", index=False)

In [47]:
# Read the saved merged dataframe
merged_df = pd.read_csv("data/merged_steam_data.csv")
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41170 entries, 0 to 41169
Data columns (total 43 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   App ID                      41170 non-null  int64  
 1   Name                        41170 non-null  object 
 2   Required age                41170 non-null  int64  
 3   DLC count                   41170 non-null  int64  
 4   Detailed description        41123 non-null  object 
 5   About the game              41113 non-null  object 
 6   Short description           41156 non-null  object 
 7   Reviews                     7920 non-null   object 
 8   Windows                     41170 non-null  bool   
 9   Mac                         41170 non-null  bool   
 10  Linux                       41170 non-null  bool   
 11  Metacritic score            41170 non-null  int64  
 12  Achievements                41170 non-null  int64  
 13  Recommendations             411

## 